# 실습 1 신용카드 이상 탐지

데이터 준비
실습에 사용될 데이터는 Kaggle의 Credit Card Fraud Detection 데이터셋입니다. 이 데이터셋은 거래의 시간, 금액과 함께 28개의 PCA 변환된 특성들을 포함하고 있습니다. 'Class' 레이블은 사기 거래를 나타내는 1과 정상 거래를 나타내는 0으로 구분됩니다.

데이터를 불러오고, 전처리하는 기본적인 코드는 아래와 같습니다:

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# 데이터를 불러옵니다.
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [ ]:
# target 데이터(class)의 분포를 확인합니다.
print("Class Distribution:")
print(data['Class'].value_counts())

# 위 모든 feature를 사용해서, class를 예측
target = data['Class'].to_numpy()
data = data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18',
             'V19','V20','V21','V22','V23','V24','V25','V26','V27','V28']].to_numpy()


Class Distribution:
Class
0    284315
1       492
Name: count, dtype: int64


In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
훈련용_data, 테스트용_data, 훈련용_target, 테스트용_target = train_test_split(data, target, test_size=0.2, random_state = 40, stratify=y)

# 데이터 표준화 작업을 실시합니다,
ss = StandardScaler()
ss.fit(훈련용_data)

표준화_훈련용_data = ss.transform(훈련용_data)
표준화_테스트용_data = ss.transform(테스트용_data)

In [ ]:
# 로지스틱 회귀 모델을 생성하고 학습합니다.
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
lr = LogisticRegression(C=5, max_iter=200)
lr.fit(표준화_훈련용_data, 훈련용_target)

# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
print(테스트용_data)
y_pred_test = lr.predict(표준화_테스트용_data)
print(y_pred_test)
# 정확도, 정밀도, F1 Score를 계산 및 출력합니다.
acc = accuracy_score(테스트용_target, y_pred_test)
prec = precision_score(테스트용_target, y_pred_test)
f1score = f1_score(테스트용_target, y_pred_test)
print(acc)
print(prec)
print(f1score)
# AUC 점수를 계산합니다.
lr_auc_score = roc_auc_score(테스트용_target, lr.predict_proba(표준화_테스트용_data)[:, 1])
print("Logistic Regression AUC score:", lr_auc_score)

[[ 1.13383486e+00 -6.67208929e-01  2.58635273e-01 ... -3.27569789e-01
   1.24504869e-02  2.77956292e-02]
 [-1.13069506e+00  8.12982397e-01  8.92485042e-01 ... -4.12572150e-01
  -4.37357056e-01 -2.20802233e-01]
 [ 2.04538850e+00  3.92553816e-03 -1.19856028e+00 ...  2.52676894e+00
  -2.05892242e-01 -1.04809019e-01]
 ...
 [ 1.15291695e+00  6.50888419e-01  8.23124580e-02 ...  6.50926580e-02
  -6.31823766e-02 -1.41751680e-02]
 [ 1.21661870e+00 -2.03749325e-01  6.02167554e-01 ...  1.07278933e+00
  -6.96084024e-02 -1.18421105e-03]
 [ 1.44284134e-01  1.09701381e+00 -1.27984818e+00 ...  1.35989147e-01
  -1.33632375e-01 -1.89512122e-02]]
[0 0 0 ... 0 0 0]
0.9992626663389628
0.8873239436619719
0.75
Logistic Regression AUC score: 0.9758404105944536


## Decision Tree 로 직접해보기

In [ ]:
# 결정 트리 모델을 생성하고 학습합니다.
dt = DecisionTreeClassifier()
dt.fit(훈련용_data, 훈련용_target)
# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
print(dt.score(훈련용_data, 훈련용_target))
print(dt.score(테스트용_data, 테스트용_target))

# AUC 점수를 계산합니다.
dt_auc_score = roc_auc_score(테스트용_target, dt.predict_proba(테스트용_data)[:, 1])
print("Decision Tree AUC score:", dt_auc_score)

1.0
0.9992802219023208
Decision Tree AUC score: 0.9070142614856492


## Random Forest 로 해보기

In [ ]:
# 랜덤 포레스트 모델을 생성하고 학습합니다.
rf = RandomForestClassifier()
rf.fit(훈련용_data, 훈련용_target)
# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
print(rf.score(훈련용_data, 훈련용_target))
print(rf.score(테스트용_data, 테스트용_target))
# AUC 점수를 계산합니다.
rf_auc_score = roc_auc_score(테스트용_target, rf.predict_proba(테스트용_data)[:, 1])
print("Decision Tree AUC score:", rf_auc_score)

1.0
0.9996664442961974
Decision Tree AUC score: 0.9732639521529105


## 퀴즈) SVM 사용해보기

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(훈련용_data, 테스트용_data)
y_pred = svc.predict(훈련용_target)
y_pred_proba = svc.predict_proba(훈련용_target)

print(y_pred_proba)

# 가장 AUC점수가 높았던 모델 GridSearch로 튜닝하기

In [ ]:
from sklearn.model_selection import GridSearchCV
parms = {'max_depth':range(1,10), 'min_samples_split':range(2,20)}

gs = GridSearchCV(RandomForestClassifier(random_state=50), parms, n_jobs=-1)
gs.fit(훈련용_data, 훈련용_target)
print(gs.best_params_)
df = gs.best_estimator_
print(df.score(훈련용_data, 훈련용_target))
print(df.score(테스트용_data, 테스트용_target))

KeyboardInterrupt: 